### Algorithm to Approximate Stabilizer Rank
* https://arxiv.org/abs/1711.07848 - On the Geometry of Stabilizer States
* $\underline{\text{First Approach - Approximating an arbitrary quantum state with a single stabilizer state (Section 4.1)}}$
    * Outline of approach:
        * Begin with arbitrary state - 
        $$\ket{\psi} = \sum_{i = 1}^{k} \alpha_i \ket{s_i}$$
        * Start at stabilizer state $\ket{s_m}$ such that it has largest $|\alpha_i|$
        * At each iteration, evaluate $N(\ket{s_m})$ (nearest neighbors of $\ket{s_m}$) by computing 
        $$\max_{\ket{\phi} \in N(\ket{s_m})} |\bra{\phi}\ket{\psi}|$$
* $\underline{\text{Second Approach - Orthogonalization of Stabilizer States (Section 5.3)}}$

In [1]:
import stim
import numpy as np
import qiskit
import itertools
import nbimporter

from NonCliff_1Meas_V4 import *

from qiskit.quantum_info import Pauli, PauliList, StabilizerState
from typing import List, Tuple

In [2]:
def get_orthogonal_vecs(used_neighbors: np.array, neighbor_list: List) -> List:
    """ 
    Finds the set of neighboring vectors that are orthogonal to 
    vectors that have already been used as optimal neighbors

    Parameters:
    -----------
    used_neighbors - List of previous optimal choice of nearest neighbors
    neighbor_list - List of nearest neighbors

    Returns:
    --------
    List of possible options for next optimal neighbor
    """
    option_list = []
    if (used_neighbors == []):
        option_list = neighbor_list 
    else:
        option_list = [neighbor for neighbor in neighbor_list for used in used_neighbors if (np.abs(np.vdot(used, neighbor)) == 0)]
    return option_list

def state_max_coeff(eig_vec: np.array, basis_vecs: List) -> int:
    """ 
    Finds the constituent stabilizer state (computational basis state) with 
    largest coefficient amplitude

    Parameters:
    -----------
    eig_vec - Eigenvector 
    basis_vecs - List of basis vectors

    Returns:
    --------
    Number corresponding to basis vector with largest coefficient amplitude
    """
    max_pos = np.argmax(np.abs(eig_vec))
    return max_pos

def comp_base_vecs(num_qubits: int) -> List:
    """
    Return all computational basis vectors for a given number of qubits

    Parameters:
    -----------
    num_qubits - Number of qubits

    Returns:
    --------
    Basis vectors in list
    """
    basis_vecs = []
    for i in range(2**num_qubits):
        start_vec = np.zeros((2**num_qubits,), dtype=complex)
        start_vec[i] = 1+0j
        basis_vecs.append(start_vec)
    
    return basis_vecs

def find_nearest_neighbors(stab_state: np.array, basis_vecs: List) -> List:
    """ 
    Find and return all nearest neighbor states to chosen stabilizer state 'stab_state'

    Parameters:
    -----------
    stab_state - Stabilizer state for which we wish to find nearest neighbors
    basis_vecs - Computational Basis vectors

    Returns:
    --------
    List of nearest neighbor states
    """
    phases = [1, -1, 1j, -1j]
    norm_factor = np.sqrt(2)
    nearest_neighbors = []
    for p in phases:
        for vec in basis_vecs:
            if (np.array_equal(stab_state, vec)):
                continue
            else:
                neighbor_state = np.add(stab_state, p * vec)/norm_factor 
                nearest_neighbors.append(neighbor_state)
    
    nearest_neighbors.append(stab_state)
    return nearest_neighbors

def optimal_nearest_neighbor(eig_vec: np.array, nearest_neighbors: List, chosen_neighbors: List) -> np.array:
    """ 
    Find nearest neighbor stabilizer state with maximum inner product

    Parameters:
    -----------
    eig_vec - Eigenvector we are trying to approximate
    nearest_neighbors - Nearest neighbors list
    chosen_neighbors - optimal neighbors chosen in previous iterations

    Returns:
    --------
    Single stabilizer state that is closest to 'eig_vec'
    """
    neighbor_options = get_orthogonal_vecs(chosen_neighbors, nearest_neighbors)
    #neighbor_options = nearest_neighbors
    return nearest_neighbors[np.argmax([np.abs(np.vdot(eig_vec, neighbor)) for neighbor in neighbor_options])]

In [10]:
def generate_binary_nums(num_qubits: int) -> List:
    """ 
    Generate binary numbers from 0 up to 'num_qubits'

    Parameters:
    -----------
    num_qubits - Number of qubits

    Returns:
    --------
    List of binary numbers
    """
    return [bin(i)[2:].zfill(num_qubits) for i in range(2**num_qubits)]

def generate_all_paulis(num_qubits: int) -> List: 
    """ 
    Generate all possible paulis given 'num_qubits' 

    Parameters:
    -----------
    num_qubits - Number of qubits

    Returns:
    --------
    List of all paulis
    """
    phases = [0,1,2,3]
    pauli_z_str = generate_binary_nums(num_qubits)
    pauli_z_bool = [np.array([int(i) for i in elem], dtype=bool) for elem in pauli_z_str]
    pauli_x_str = generate_binary_nums(num_qubits)
    pauli_x_bool = [np.array([int(i) for i in elem], dtype=bool) for elem in pauli_x_str]
    all_paulis_strings = [elem for elem in itertools.product(*[pauli_z_bool, pauli_x_bool])]
    all_paulis = [Pauli((p[0], p[1], phase)) for p in all_paulis_strings for phase in phases]
    return all_paulis

def find_nearest_neighbors_general(num_qubits: int, stab_state: np.array) -> List:
    """ 
    Construct list of all nearest neighbor stabilizer states for arbitrary stabilizer state.

    Parameters:
    -----------
    stab_state - Stabilizer state for which we wish to find neighbors

    Returns:
    --------
    List of nearest neighbors
    """
    bias = 1/np.sqrt(2)
    all_paulis = generate_all_paulis(num_qubits)
    new_stabs = [p.to_matrix() @ stab_state for p in all_paulis]
    new_stabs_orthogonal = [stab for stab in new_stabs if np.vdot(stab_state, stab) == 0]
    neighbors = [np.add(stab_state, stab) * bias for stab in new_stabs_orthogonal]
    neighbors_dict = {stab.tobytes(): stab for stab in neighbors}
    # return list(neighbors_dict.values())
    final_neighbors = list(neighbors_dict.values())
    final_neighbors.append(stab_state)
    return final_neighbors


def iterated_optimal_nearest_neighbor(num_qubits:int, eig_vec:np.array, init_stab_state: np.array) -> np.array:
    """ 
    Iteratively compute the optimal nearest neighbor whose inner product with 'eig_vec' is maximal

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector we wish to approximate
    init_stab_state - Initial stabilizer state where we start our search from 

    Returns:
    --------
    Optimal single state that best approximates 'eig_vec'
    """
    num_iterations = 2 ** num_qubits
    curr_stab_state = init_stab_state
    max_inner_prod = 0
    curr_best_stab_state = None
    for _ in range(num_iterations):
        curr_neighbors = find_nearest_neighbors_general(num_qubits, curr_stab_state)
        #print("The current neighbor set is: " + str(curr_neighbors))

        #optimal_neighbor = curr_neighbors[np.argmax([np.abs(np.vdot(neighbor, eig_vec)) for neighbor in curr_neighbors])]
        optimal_neighbor = curr_neighbors[np.argmax([np.vdot(neighbor, eig_vec) for neighbor in curr_neighbors])]

        curr_stab_state = optimal_neighbor 
        #print("The current stabilizer state approx is: " + str(curr_stab_state))

        #curr_inner_prod = np.abs(np.vdot(curr_stab_state, eig_vec))
        curr_inner_prod = np.vdot(curr_stab_state, eig_vec)

        if (curr_inner_prod > max_inner_prod):
            max_inner_prod = curr_inner_prod 
            curr_best_stab_state = curr_stab_state
        #print("The current inner product between the current stabilizer state approximation and the eigenvector is: " + 
        #str(curr_inner_prod))
        #print("\n")

    #print(curr_best_stab_state)
    #print(max_inner_prod)
    return curr_best_stab_state, max_inner_prod

In [20]:
num_wit_qubits = 25 # variable 'n' in paper
# For now, what the witness qubits are initialized to be doesn't matter (I THINK)
# as we are trying to minimize Val, given some optimal input states.
wit_qubits = np.zeros(num_wit_qubits, dtype=complex)

num_anc_qubits = 0 # variable 'm' in paper
anc_qubits = np.zeros(num_anc_qubits, dtype=complex)

num_t_gates = 2 # variable 't' in paper

num_meas_qubits = 1 # variable 'k' in paper
total_qubits = num_wit_qubits + num_anc_qubits

num_layers = 20

assert(num_layers > 0)

In [21]:
p_sum, _ = sim_circ(total_qubits, num_layers, num_t_gates)
succinct_p_sum = p_sum.succinct_p_sum(total_qubits)
min_eigvec = succinct_p_sum.compute_min_eig_eigvec(len(p_sum.get_bases()))[1]
new_total_qubits = len(succinct_p_sum.get_sum()[0].get_pauli_str(succinct_p_sum.get_sum()[0].get_pauli()))
print(succinct_p_sum)
print(list(min_eigvec))

[(0.7071067811865475+0j) ZII] + [(0.4999999999999999-0j) XZI] + [(-0.4999999999999999+0j) XXI]
[(-0.3826834323650897-0j), (-0-0j), (-0-0j), (-0-0j), (0.6532814824381883+0j), (-0-0j), (-0.6532814824381883-0j), (-0-0j)]


In [66]:
fin_stab_approx = np.zeros((2**new_total_qubits,), dtype=complex)
min_eigvec_cp = min_eigvec.copy()
basis_vecs = comp_base_vecs(new_total_qubits)
stab_state = basis_vecs[state_max_coeff(min_eigvec, basis_vecs)]
neighbor_list = find_nearest_neighbors(stab_state, basis_vecs)
optimal_neighbor = optimal_nearest_neighbor(min_eigvec, neighbor_list, [])
print(optimal_neighbor)
print(np.abs(np.vdot(optimal_neighbor, min_eigvec)))



[0.+0.j 1.+0.j]
1.0


In [11]:
def stab_rank_approx_one(num_qubits: int, eig_vec: np.array) -> int:
    """ 
    Approximate the stabilizer rank (using the first approach)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0
    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    used_neighbors = []
    init_state = eig_vec.copy() # Make a copy of the original eigenvector
    final_stab_approx = np.zeros((2**num_qubits,), dtype=complex) # Keep track of the final approximation
    for _ in range(2**num_qubits):
        print("The eigenvector currently looks like: " + str(eig_vec))

        max_coeff = state_max_coeff(eig_vec, basis_vecs)
        #used_basis_vecs.append(max_coeff)
        stab_state = basis_vecs[max_coeff] # Figure out which stabilizer state to find neighbors for first
        neighbors = find_nearest_neighbors(stab_state, basis_vecs) # Find all nearest neighbors of 'stab_state'
        optimal_neighbor = optimal_nearest_neighbor(eig_vec, neighbors, used_neighbors) # Find optimal nearest neighbor of 'stab_state'
        used_neighbors.append(optimal_neighbor)
        print("The optimal neighbor for the current state is: " + str(optimal_neighbor))

        # Subtract optimal neighbor component from eigenvector
        # Then normalize state
        eig_vec = np.subtract(eig_vec, optimal_neighbor)
        eig_vec = eig_vec/np.linalg.norm(eig_vec)

        # Add to optimal neighbor component to 'final_stab_approx'
        # Then normalize state 
        final_stab_approx = np.add(final_stab_approx, optimal_neighbor)
        final_stab_approx = final_stab_approx/np.linalg.norm(final_stab_approx)
        print("The approximated stabilizer state so far is: " + str(final_stab_approx))

        # Check to see how inner product between 'final_stab_approx' and 
        # 'eig_vec' progresses 
        print("The inner product between approximation and actual is: " + str(np.abs(np.vdot(final_stab_approx, init_state))))
        print("\n")
        approx_stab_rank += 1

    return approx_stab_rank

In [12]:
def stab_rank_approx_two(num_qubits: int, eig_vec: np.array) -> int:
    """ 
    Approximate the stabilizer rank (2nd attempt)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0
    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    used_neighbors = []
    init_state = eig_vec.copy() # Make a copy of the original eigenvector
    final_stab_approx = np.zeros((2**num_qubits,), dtype=complex) # Keep track of the final approximation
    for _ in range(2**num_qubits):
        print("The eigenvector currently looks like: " + str(eig_vec))
        #print("The current norm of the eigenvector is: " + str(np.linalg.norm(eig_vec)))

        # Find basis state with maximum coefficient
        max_coeff = state_max_coeff(eig_vec, basis_vecs)
        stab_state = basis_vecs[max_coeff]

        # Compute neighbors for 'stab_state' and retrieve optimal neighbor
        neighbors = find_nearest_neighbors(stab_state, basis_vecs)
        optimal_neighbor = optimal_nearest_neighbor(eig_vec, neighbors, used_neighbors)
        used_neighbors.append(optimal_neighbor)
        print("The optimal neighbor for the current state is: " + str(optimal_neighbor))

        # Subtract optimal neighbor from eigenvector 
        optimal_neighbor = (optimal_neighbor/np.linalg.norm(optimal_neighbor)) * np.linalg.norm(eig_vec)
        eig_vec = np.subtract(eig_vec, optimal_neighbor)

        # Add to optimal neighbor component to 'final_stab_approx'
        # Then normalize state 
        final_stab_approx = np.add(final_stab_approx, optimal_neighbor)
        final_stab_approx = final_stab_approx/np.linalg.norm(final_stab_approx)
        print("The approximated stabilizer state so far is: " + str(final_stab_approx))

        # Check to see how inner product between 'final_stab_approx' and 
        # 'eig_vec' progresses 
        print("The inner product between approximation and actual is: " + str(np.abs(np.vdot(final_stab_approx, init_state))))
        print("\n")
        approx_stab_rank += 1

        if (np.abs(np.vdot(final_stab_approx, init_state)) == 1.0):
            break

    return approx_stab_rank

In [13]:
def one_state_stab_approx(num_qubits:int, eig_vec: np.array) -> np.array:
    """ 
    Find best (?) single stabilizer state approximation for eigen vector 

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector we wish to approximate

    Returns:
    --------
    Best (?) single stabilizer state that approximates eigenvector
    """

    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    init_state = eig_vec.copy() # Make a copy of the original eigenvector
    #final_approx_state = np.zeros((2**num_qubits,), dtype=complex) # Where result for best stab state approx is stored
    
    # Find basis state with maximum coefficient
    max_coeff = state_max_coeff(eig_vec, basis_vecs)
    init_stab_state = basis_vecs[max_coeff]

    # Compute optimal nearest stab state in iterated fashion
    final_approx_state, max_inner_prod = iterated_optimal_nearest_neighbor(num_qubits, eig_vec, init_stab_state)

    return final_approx_state, max_inner_prod

In [14]:
def stab_rank_approx_three(num_qubits: int, eig_vec: np.array) -> Tuple:
    """ 
    Approximate the stabilizer rank (using approach discussed)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0 
    approx_vec = np.zeros(len(min_eigvec), dtype=complex) # Vector we wish to construct to approximate 'eig_vec'
    used_states = [] # Set of stabiilzer states that have been used
    prec_param = 0.04 # Precision parameter to see how close dot prod of 'approx_vec' and 'eig_vec' is

    # Find initial closest stabilizer state to our arbitrary state
    s, inner_prod = one_state_stab_approx(num_qubits, eig_vec) 
    used_states.append(s)
    if (np.array_equal(s,eig_vec)):
        approx_stab_rank += 1
        approx_vec += s
        return approx_stab_rank, approx_vec
    copy_vec = eig_vec.copy()
    while(1 - inner_prod > prec_param or approx_stab_rank < 2 ** num_qubits):
        s_norm = np.linalg.norm(s)
        proj_vec = (np.vdot(s, eig_vec)/s_norm**2) * s 
        print("proj_vec: " + str(proj_vec))
        approx_vec += proj_vec 
        approx_stab_rank += 1
        
        e_norm = np.linalg.norm(eig_vec - proj_vec)
        eig_vec = (eig_vec - proj_vec)/e_norm 
        print("eig_vec: " + str(eig_vec))
        s, s_inner_prod = one_state_stab_approx(num_qubits, eig_vec)
        print("stab state: " + str(s))
        used_states.append(s)
        #print("Approx so far: " + str(approx_vec))
        inner_prod = np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec)

    approx_vec = approx_vec / np.linalg.norm(approx_vec)
    return approx_stab_rank, approx_vec 

In [25]:
def stab_rank_approx_four(num_qubits: int, eig_vec: np.array) -> Tuple:
    """ 
    Approximate the stabilizer rank (using approach discussed)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0 
    approx_vec = np.zeros(len(min_eigvec), dtype=complex) # Vector we wish to construct to approximate 'eig_vec'
    prec_param = 0.04 # Precision parameter to see how close dot prod of 'approx_vec' and 'eig_vec' is
    norm_factor = 0
    is_approx = False # While loop termination condition

    copy_vec = eig_vec.copy()
    while (is_approx == False):
        # Calculate nearest stabilizer state
        s, s_inner_prod = one_state_stab_approx(num_qubits, eig_vec)
        print("Stab: " + str(list(s)))

        # Project 'eig_vec' on s
        s_norm = np.linalg.norm(s)
        s_coeff = np.vdot(s, eig_vec)
        if (norm_factor == 0):
            proj_vec = (s_coeff/s_norm**2) * s
        else:
            proj_vec = norm_factor * (s_coeff/s_norm**2) * s

        # Subtract projection of 'eig_vec' on s from 'eig_vec'
        unnorm_eig_vec = eig_vec - proj_vec 
        norm_factor = np.linalg.norm(unnorm_eig_vec)
        eig_vec = unnorm_eig_vec / norm_factor

        # Add projection to approximation and update stab rank
        approx_vec += proj_vec
        approx_stab_rank += 1

        # Check closeness of approximation to original 'eig_vec'
        print(np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec))
        if (np.abs(1 - np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec)) <= prec_param):
            is_approx = True 
        #print(np.linalg.norm((approx_vec / np.linalg.norm(approx_vec)) - copy_vec))
        #if (np.linalg.norm((approx_vec / np.linalg.norm(approx_vec)) - copy_vec) <= prec_param):
        #    is_approx = True

        if (approx_stab_rank == 2 ** num_qubits):
            break

        print(list(approx_vec))
        
    approx_vec = approx_vec / np.linalg.norm(approx_vec)
    
    return approx_stab_rank, approx_vec


In [16]:
def calc_proj_vec(used_stab_states: List, eig_vec: np.array, norm_factor: complex):
    """ 
    Calculate sum of all projection vectors of eig_vec onto chosen stabilizer states

    Params:
    -------
    used_stab_states - Set of stabilizer states that have been used already
    eig_vec - Eigenvector we are trying to project onto our stabilizer states
    norm_factor - Normalization factor for 'eig_vec'
    
    """
    proj_vec = 0
    stab_state_norms = [np.linalg.norm(s) for s in used_stab_states]
    stab_state_coeffs = [np.vdot(s, eig_vec) for s in used_stab_states]
    for i in range(len(used_stab_states)):
        proj_vec += (stab_state_coeffs[i] / stab_state_norms[i] ** 2) * used_stab_states[i]
    if (norm_factor != 0):
        proj_vec *= norm_factor 

    return proj_vec



def stab_rank_approx_five(num_qubits: int, eig_vec: np.array) -> Tuple:
    """ 
    Approximate the stabilizer rank (using approach discussed)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0
    approx_vec = np.zeros(len(min_eigvec), dtype=complex) # Vector we wish to construct to approximate 'eig_vec'
    prec_param = 0.04 # Precision parameter to see how close dot prod of 'approx_vec' and 'eig_vec' is
    norm_factor = 0
    is_approx = False # While loop termination condition

    used_stab_states = []

    copy_vec = eig_vec.copy()
    while (is_approx == False):
        # Calculate nearest stabilizer state
        s, _ = one_state_stab_approx(num_qubits, eig_vec)
        used_stab_states.append(s)

        # Project 'eig_vec' on s
        proj_vec = calc_proj_vec(used_stab_states, eig_vec, norm_factor)

        # Subtract projection of 'eig_vec' on s from 'eig_vec' 
        unnorm_eig_vec = eig_vec - proj_vec 
        norm_factor = np.linalg.norm(unnorm_eig_vec)
        eig_vec = unnorm_eig_vec / norm_factor

        # Add projection to approximation and update stab rank 
        approx_vec += proj_vec 
        approx_stab_rank += 1

        # Check closeness of approximatino to original 'eig_vec' 
        print(np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec))
        if (np.abs(1 - np.vdot(approx_vec / np.linalg.norm(approx_vec), copy_vec)) <= prec_param):
            is_approx = True 
        #print(np.linalg.norm((approx_vec / np.linalg.norm(approx_vec)) - copy_vec))
        #if (np.linalg.norm((approx_vec / np.linalg.norm(approx_vec)) - copy_vec) <= prec_param):
        #    is_approx = True

        if (approx_stab_rank == 2 ** num_qubits):
            break 

        print(approx_vec) 

    approx_vec = approx_vec / np.linalg.norm(approx_vec)
    
    return approx_stab_rank, approx_vec

In [26]:
approx_stab_rank, approx_vec = stab_rank_approx_four(new_total_qubits, min_eigvec)
print(approx_stab_rank, approx_vec)


Stab: [0j, 0j, 0j, 0j, (0.7071067811865475+0j), 0j, (-0.7071067811865475+0j), 0j]
(0.9238795325112868+0j)
[0j, 0j, 0j, 0j, (0.6532814824381884+0j), 0j, (-0.6532814824381884+0j), 0j]
Stab: [(-0.9999999999999997+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j]
(1.0000000000000002+0j)
[(-0.3826834323650897+0j), 0j, 0j, 0j, (0.6532814824381884+0j), 0j, (-0.6532814824381884+0j), 0j]
2 [-0.38268343+0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.65328148+0.j  0.        +0.j -0.65328148+0.j  0.        +0.j]


In [207]:
print(min_eigvec)
print(np.vdot(min_eigvec, approx_vec))

[-1.74604928e-04+3.13179054e-04j  2.95492580e-04-9.22262234e-05j
 -3.13179054e-04-1.74604928e-04j  9.22262234e-05+2.95492580e-04j
 -6.93969347e-03-2.61999279e-03j  1.84452447e-04-7.06775686e-01j
  2.27078293e-03-6.31333536e-03j  7.07366672e-01+0.00000000e+00j]
(0.9999496593387529+2.710505431213761e-20j)


In [176]:
old_s = np.array([[0.70710678+0.j, 0.        +0.j, 0.70710678+0.j, 0.        +0.j]])
e_vec = np.array([[(0.9238795325112867+0j), (-0-0j), (0.3826834323650898-0j), (-0-0j)]])
print(e_vec)
old_s_norm = np.linalg.norm(old_s)
proj_e_vec_s = (np.vdot(e_vec, old_s)/old_s_norm**2) * old_s 

e_vec = e_vec - proj_e_vec_s
print(e_vec)
e_norm = np.linalg.norm(e_vec)
print(e_norm)
print(e_vec / e_norm)
s, inner_prod = one_state_stab_approx(2, e_vec / e_norm)
s

[[0.92387953+0.j 0.        +0.j 0.38268343+0.j 0.        +0.j]]
[[ 0.27059805+0.j  0.        +0.j -0.27059805+0.j  0.        +0.j]]
0.3826834323650897
[[ 0.70710678+0.j  0.        +0.j -0.70710678+0.j  0.        +0.j]]


array([ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,  0.        +0.j])

### Approaches #1 and #2
* Based on the above results, testing for various eigenvectors for Pauli Sums, we find that this algorithm gets its best approximation in the first iteration
* Past one iteration, the approximations move too far away from the true eigenvector

### Approach #3: Best Stabilizer State
* Able to find stabilizer state that is as close as possible to eigenvector
* Next step, can, one orthogonal stabilizer state at a time, project eigenvector onto spaces spanned by stabilizer vectors in order to find linear combination that embodies stabilizer rank